<h1>Capstone Project: Private Property Neighborhoods in Singapore</h1>
<h1>This notebook will be mainly used for the capstone project</h1>
<h4>Includes using location data in determining the suitable neighbourhoods, Clustering the neighbourhoods in Toronto</h4>


In [87]:
import pandas as pd
import numpy as np
import json

In [88]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [89]:
!pip install geopy
!pip install folium==0.5.0
!pip install geocoder
import urllib.request
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes

import folium # map rendering library
import requests
import geocoder



Solving environment: done

# All requested packages already installed.



In the cell above I have used BeautifulSoup to extract data from the wikipedia website



In [90]:
CLIENT_ID = 'K4YZ0DCCW2EWL5RQQTVEP0FU4C43H3JOWDIYETH2XDMSN45C' # your Foursquare ID
CLIENT_SECRET = 'YK5NRWVO4GW0BEUMRRL1U0DDRB3M1J2CNTCKOVKWDC4YZTYX' # your Foursquare Secret
ACCESS_TOKEN = 'RTNMBNOSCAQ0UFUNRNVII0FDSNDEB0ZFIV4DPZW1FR3LHV1Y' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K4YZ0DCCW2EWL5RQQTVEP0FU4C43H3JOWDIYETH2XDMSN45C
CLIENT_SECRET:YK5NRWVO4GW0BEUMRRL1U0DDRB3M1J2CNTCKOVKWDC4YZTYX


In [92]:
address = 'Singapore, Singapore'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Singapore%2C+Singapore&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
# Scrape wikipedia page




In [94]:
df1 =pd.read_csv("/Users/joshualeow/Desktop/DataCondo/DataPrivate1.csv")
df2 =pd.read_csv("/Users/joshualeow/Desktop/DataCondo/DataPrivate2.csv")
df3 =pd.read_csv("/Users/joshualeow/Desktop/DataCondo/DataPrivate3.csv")
df4 =pd.read_csv("/Users/joshualeow/Desktop/DataCondo/DataPrivate4.csv")
df5 =pd.read_csv("/Users/joshualeow/Desktop/DataCondo/DataPrivate5.csv")
df6 =pd.read_csv("/Users/joshualeow/Desktop/DataCondo/DataPrivate6.csv")

frames = [df1, df2, df3, df4, df5, df6]

df_merged = pd.concat(frames)

df_merged.columns = df_merged.iloc[0]
df_merged = df_merged[1:]


df_merged.rename(columns={'level_0': 'S/N', 'level_1': 'ProjectName'}, inplace=True)
df_merged = df_merged.reset_index(drop=False).rename(columns={'level_0':'S/N', 'level_1':'ProjectName', 'level_2':'StreetName','level_3':'Type','level_4':'PostalDistrict','level_5':'MarketSegment','level_6':'Tenure','level_7':'TypeOfSale','level_8':'NoUnits','level_9':'Price','level_10':'NettPrice','level_11':'Sqft','level_12':'TypeOfArea','level_13':'FloorLevel','level_14':'UnitPrice','level_15':'DateOfSale'})


print(df_merged.columns.tolist())

df_merged.iloc[19450:19460]

['S/N', 'ProjectName', 'StreetName', 'Type', 'PostalDistrict', 'MarketSegment', 'Tenure', 'TypeOfSale', 'NoUnits', 'Price', 'NettPrice', 'Sqft', 'TypeOfArea', 'FloorLevel', 'UnitPrice', 'Date of Sale']


"(S/N, Project Name, Street Name, Type, Postal District, Market Segment, Tenure, Type of Sale, No. of Units, Price ($), Nett Price ($), Area (Sqft), Type of Area, Floor Level, Unit Price ($psf))",S/N,ProjectName,StreetName,Type,PostalDistrict,MarketSegment,Tenure,TypeOfSale,NoUnits,Price,NettPrice,Sqft,TypeOfArea,FloorLevel,UnitPrice,Date of Sale
19450,3,LEEDON GREEN,LEEDON HEIGHTS,Condominium,10,CCR,Freehold,New Sale,1,1699700,-,710,Strata,01 to 05,2393,May-2021
19451,4,HAUS ON HANDY,HANDY ROAD,Condominium,09,CCR,99 yrs lease commencing from 2018,New Sale,1,1550896,-,592,Strata,06 to 10,2620,May-2021
19452,5,LEEDON GREEN,LEEDON HEIGHTS,Condominium,10,CCR,Freehold,New Sale,1,1809400,-,700,Strata,06 to 10,2586,May-2021
19453,6,D'LEEDON,LEEDON HEIGHTS,Condominium,10,CCR,99 yrs lease commencing from 2010,Resale,1,2000000,-,1335,Strata,06 to 10,1498,May-2021
19454,7,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,1806000,-,657,Strata,16 to 20,2751,May-2021
19455,8,DUCHESS MANOR,DUCHESS ROAD,Condominium,10,CCR,999 yrs lease commencing from 1875,Resale,1,1695000,-,807,Strata,01 to 05,2100,May-2021
19456,9,PARK NOVA,TOMLINSON ROAD,Apartment,10,CCR,Freehold,New Sale,1,14458000,-,2896,Strata,16 to 20,4993,May-2021
19457,10,PARK NOVA,TOMLINSON ROAD,Apartment,10,CCR,Freehold,New Sale,1,14618000,-,2896,Strata,16 to 20,5048,May-2021
19458,11,PARK NOVA,TOMLINSON ROAD,Apartment,10,CCR,Freehold,New Sale,1,14768000,-,2906,Strata,16 to 20,5081,May-2021
19459,12,PARK NOVA,TOMLINSON ROAD,Apartment,10,CCR,Freehold,New Sale,1,34438000,-,5899,Strata,16 to 20,5838,May-2021


In [93]:

list_of_sn = df_merged['StreetName'].unique().tolist()
list_of_sn

['STIRLING ROAD',
 'CLEMENTI AVENUE 1',
 'CHIN SWEE ROAD',
 'SILAT AVENUE',
 'BERNAM STREET',
 'NORMANTON PARK',
 'SLIM BARRACKS RISE',
 'FABER WALK',
 'JALAN LEMPENG',
 'SPOTTISWOODE PARK ROAD',
 'WEST COAST CRESCENT',
 'WEST COAST VALE',
 'PEARL BANK',
 'ENG HOON STREET',
 'ALEXANDRA ROAD',
 'KEPPEL BAY DRIVE',
 'PASIR PANJANG HILL',
 'MOUNT FABER ROAD',
 'HARBOURFRONT AVENUE',
 'COVE WAY',
 'MISTRI ROAD',
 'CANTONMENT ROAD',
 'WEST COAST WAY',
 'PRINCE CHARLES CRESCENT',
 'COMMONWEALTH AVENUE',
 'ONE-NORTH GATEWAY',
 'WEST COAST ROAD',
 'DOVER RISE',
 'DUNDEE ROAD',
 'INDUS ROAD',
 'SOUTH BUONA VISTA ROAD',
 'WEST COAST WALK',
 'ALEXANDRA VIEW',
 'OCEAN DRIVE',
 'JIAK KIM STREET',
 'SHENTON WAY',
 'GOPENG STREET',
 'PARK ROAD',
 'CLUB STREET',
 'WEST COAST PARK',
 'MARINA BOULEVARD',
 'WALLICH STREET',
 'DEPOT ROAD',
 'ENGGOR STREET',
 'TELOK BLANGAH ROAD',
 'COVE DRIVE',
 'HAVELOCK ROAD',
 'WEST COAST RISE',
 'ROCHESTER DRIVE',
 'PASIR PANJANG ROAD',
 'KEPPEL BAY VIEW',
 'WEST COAS

In [95]:
prop_lat = []
prop_long = []
for i in range(0, len(list_of_sn)):
    query_address = list_of_sn[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_prop=json.loads(resp.content)
    
    if data_prop['found'] != 0:
        prop_lat.append(data_prop["results"][0]["LATITUDE"])
        prop_long.append(data_prop["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_prop['results'][0]['LATITUDE'] +" Long: "+data_prop['results'][0]['LONGITUDE'])

    else:
        prop_lat.append('NotFound')
        prop_long.append('NotFound')
        print ("No Results")
        
# With reference from https://towardsdatascience.com/working-with-apis-in-data-science-explore-bit-rent-theory-in-singapores-hdb-resale-market-d7760fdfc601


STIRLING ROAD,Lat: 1.29091128182876 Long: 103.803120797076
CLEMENTI AVENUE 1,Lat: 1.30819332208405 Long: 103.767915046417
CHIN SWEE ROAD,Lat: 1.28855135333783 Long: 103.841303134744
SILAT AVENUE,Lat: 1.27707706879078 Long: 103.831425856916
BERNAM STREET,Lat: 1.27386294350519 Long: 103.843710484506
NORMANTON PARK,Lat: 1.28705408999986 Long: 103.792894551099
SLIM BARRACKS RISE,Lat: 1.3030110355336 Long: 103.78987884758
FABER WALK,Lat: 1.32147970071034 Long: 103.756050048879
JALAN LEMPENG,Lat: 1.31814179257988 Long: 103.761165466535
SPOTTISWOODE PARK ROAD,Lat: 1.27394215034034 Long: 103.835659267953
WEST COAST CRESCENT,Lat: 1.2979267311648 Long: 103.765703038497
WEST COAST VALE,Lat: 1.31827093169981 Long: 103.752817355978
PEARL BANK,Lat: 1.2829639133069 Long: 103.839787967863
ENG HOON STREET,Lat: 1.28375041465604 Long: 103.834187718541
ALEXANDRA ROAD,Lat: 1.28679161744874 Long: 103.796720921698
KEPPEL BAY DRIVE,Lat: 1.26574158473368 Long: 103.816213916683
PASIR PANJANG HILL,Lat: 1.2915421

SCOTTS ROAD,Lat: 1.30985881334205 Long: 103.834186549711
HOLLAND VILLAGE WAY,Lat: 1.31000314657577 Long: 103.794968029324
MIDDLE ROAD,Lat: 1.29973919380889 Long: 103.852196307383
HERTFORD ROAD,Lat: 1.31305644462309 Long: 103.848338802186
FLANDERS SQUARE,Lat: 1.31218127904025 Long: 103.857452938926
JERVOIS ROAD,Lat: 1.29262743689233 Long: 103.825615292632
MOUNT SINAI LANE,Lat: 1.31571971777096 Long: 103.78537397619
TANGLIN ROAD,Lat: 1.30186472795448 Long: 103.798174301014
PATERSON HILL,Lat: 1.30051355857344 Long: 103.828725574937
TAN QUEE LAN STREET,Lat: 1.29845248443496 Long: 103.85674202786
MOUNT SINAI RISE,Lat: 1.31592078058868 Long: 103.780266916966
JERVOIS CLOSE,Lat: 1.29273462317897 Long: 103.824359105277
FARRER ROAD,Lat: 1.31755039971444 Long: 103.807384282049
BUKIT TIMAH ROAD,Lat: 1.31090270967523 Long: 103.842792749529
MERGUI ROAD,Lat: 1.3172213502844 Long: 103.852101076098
CAIRNHILL ROAD,Lat: 1.30564918648762 Long: 103.837499350934
TAMAN SERASI,Lat: 1.31127896443187 Long: 103.

MOUNT ELIZABETH,Lat: 1.30669217339058 Long: 103.836183395425
TANGLIN RISE,Lat: 1.30184226862242 Long: 103.817753334705
NAPIER ROAD,Lat: 1.30506987517629 Long: 103.821462810626
HOLLAND PARK,Lat: 1.30810383760868 Long: 103.80656281694
JALAN HAJI ALIAS,Lat: 1.32302340248572 Long: 103.795186213548
PENHAS ROAD,Lat: 1.30936948081609 Long: 103.862759811631
SOMMERVILLE ESTATE ROAD,Lat: 1.31044101000472 Long: 103.806375878889
KITCHENER ROAD,Lat: 1.31053024860512 Long: 103.855657424789
SELEGIE ROAD,Lat: 1.30221805203586 Long: 103.850050439737
FERNHILL ROAD,Lat: 1.31281683141403 Long: 103.823397100293
KENT ROAD,Lat: 1.29434507610317 Long: 103.769823829533
BIRCH ROAD,Lat: 1.31120906406961 Long: 103.853723935062
SYED ALWI ROAD,Lat: 1.30634556161643 Long: 103.859160932507
SAM LEONG ROAD,Lat: 1.30940093932423 Long: 103.856829411756
CHATSWORTH ROAD,Lat: 1.30030237661746 Long: 103.821152766448
ROBIN WALK,Lat: 1.31945970129013 Long: 103.826284523197
CLIVE STREET,Lat: 1.30558593878871 Long: 103.852456825

SHAN ROAD,Lat: 1.32477217623719 Long: 103.846667321915
BRADDELL ROAD,Lat: 1.34409256877338 Long: 103.850153965874
LORONG 6 GEYLANG,Lat: 1.31022188230269 Long: 103.876641754539
HAIGSVILLE DRIVE,Lat: 1.31019826607232 Long: 103.898552263463
SIGLAP LINK,Lat: 1.30751017974257 Long: 103.928316868421
ROSE LANE,Lat: 1.31024036682965 Long: 103.893929635401
SWISS VIEW,Lat: 1.34707238320757 Long: 103.789833863894
LEICESTER ROAD,Lat: 1.33002356564995 Long: 103.86727711208
PEGU ROAD,Lat: 1.32324764631144 Long: 103.851474289537
CARPMAEL ROAD,Lat: 1.3107729355313 Long: 103.899849716962
LORONG 38 GEYLANG,Lat: 1.31422873871124 Long: 103.889162081937
LORONG 39 GEYLANG,Lat: 1.315699751972 Long: 103.890724332553
WAN THO AVENUE,Lat: 1.33269944626651 Long: 103.872216293609
LORONG 33 GEYLANG,Lat: 1.31518727098016 Long: 103.887768832797
GEYLANG ROAD,Lat: 1.31373238484063 Long: 103.886012058953
EAST COAST TERRACE,Lat: 1.31916763205584 Long: 103.95332030818
LORONG SARINA,Lat: 1.32014742772285 Long: 103.90610310

LORONG 9 GEYLANG,Lat: 1.31282503568126 Long: 103.876411144282
WATTEN VIEW,Lat: 1.3306692904189 Long: 103.808746721199
CHAPEL ROAD,Lat: 1.30620386773828 Long: 103.907634092525
WHAMPOA DRIVE,Lat: 1.32318696807729 Long: 103.853885426169
GREENWOOD AVENUE,Lat: 1.33313055800383 Long: 103.805248142049
GRAY LANE,Lat: 1.31245246175305 Long: 103.89316187328
JALAN JAMAL,Lat: 1.31343974218318 Long: 103.925206885175
SALLIM ROAD,Lat: 1.32618174082201 Long: 103.881739191849
LAVENDER STREET,Lat: 1.31337862160315 Long: 103.861165196035
FOWLIE ROAD,Lat: 1.30649466680817 Long: 103.902559416681
LORONG 18 GEYLANG,Lat: 1.3114901867856 Long: 103.880695674867
JALAN SUKA,Lat: 1.31219718173542 Long: 103.882968715437
HAIG LANE,Lat: 1.30896488513857 Long: 103.898710637523
JALAN WANGI,Lat: 1.3310151683642 Long: 103.874252005791
 13,Lat: 1.31309640589843 Long: 103.877414902097
SERANGOON NORTH AVENUE 1,Lat: 1.36995099247833 Long: 103.872312299228
TAMPINES LANE,Lat: 1.3446291757256 Long: 103.947119140058
FLORA DRIVE,

SALAM WALK,Lat: 1.32141912651159 Long: 103.950767440832
LEW LIAN VALE,Lat: 1.34934191512027 Long: 103.878019230988
JALAN PELIKAT,Lat: 1.35387046741456 Long: 103.885941782251
SERAYA CRESCENT,Lat: 1.37359918404024 Long: 103.8266633864
ROSYTH ROAD,Lat: 1.36271217675265 Long: 103.876535798161
FLORENCE ROAD,Lat: 1.36466138641777 Long: 103.884949624324
SANDILANDS ROAD,Lat: 1.36282557285688 Long: 103.878288140176
LIM AH PIN ROAD,Lat: 1.36328727955649 Long: 103.887787260083
TEOW HOCK AVENUE,Lat: 1.36208861088978 Long: 103.879772670706
SIRAT ROAD,Lat: 1.35799509876294 Long: 103.875818221287
JALAN NAUNG,Lat: 1.36953201751479 Long: 103.892819330262
UPPER CHANGI ROAD,Lat: 1.32401711963191 Long: 103.938966799723
HIGHLAND ROAD,Lat: 1.35786123164277 Long: 103.878827799133
GAMBIR WALK,Lat: 1.34444791593051 Long: 103.874989694737
JALAN TIGA RATUS,Lat: 1.3416251882966 Long: 103.959995683054
BUANGKOK GREEN,Lat: 1.37992376908674 Long: 103.887491361899
JALAN PARI BURONG,Lat: 1.33382628293841 Long: 103.9469

YISHUN AVENUE 1,Lat: 1.42036586749655 Long: 103.848068608027
GERALD DRIVE,Lat: 1.3866250558307 Long: 103.878444960025
YISHUN CLOSE,Lat: 1.43453743031668 Long: 103.844831194221
FERNVALE STREET,Lat: 1.39844225909291 Long: 103.879093940733
YISHUN AVENUE 9,Lat: 1.43162370844369 Long: 103.83984841958
KANDIS LINK,Lat: 1.45663401475902 Long: 103.837018674889
NIM ROAD,Lat: 1.38213894564358 Long: 103.863033086611
CACTUS ROAD,Lat: 1.39014657033248 Long: 103.852186885171
SEMBAWANG DRIVE,Lat: 1.45555388401848 Long: 103.81464932384
JALAN TARI SERIMPI,Lat: 1.39710020359873 Long: 103.872497067009
MIMOSA ROAD,Lat: 1.38427217652461 Long: 103.862870863743
MIMOSA CRESCENT,Lat: 1.38513290820973 Long: 103.862921421272
No Results


In [153]:
proplocation = pd.DataFrame({
    'StreetName': list_of_sn,
    'latitude': prop_lat,
    'longitude': prop_long})

proplocation.head(10)
proplocation['StreetName'].head

<bound method NDFrame.head of 0           STIRLING ROAD
1       CLEMENTI AVENUE 1
2          CHIN SWEE ROAD
3            SILAT AVENUE
4           BERNAM STREET
              ...        
835       SEMBAWANG DRIVE
836    JALAN TARI SERIMPI
837           MIMOSA ROAD
838       MIMOSA CRESCENT
839             Sub Sale 
Name: StreetName, Length: 840, dtype: object>

<h3>Now I will combine the df_merged and proplocation dataframes into one.</h3>

In [161]:
outerdata = pd.merge(df_merged, proplocation, how="outer", on=["StreetName"])

outerdata



,S/N,ProjectName,StreetName,Type,PostalDistrict,MarketSegment,Tenure,TypeOfSale,NoUnits,Price,NettPrice,Sqft,TypeOfArea,FloorLevel,UnitPrice,Date of Sale,latitude,longitude
0,1,STIRLING RESIDENCES,STIRLING ROAD,Apartment,03,RCR,99 yrs lease commencing from 2017,New Sale,1,1401000,-,635,Strata,36 to 40,2206,May-2021,1.29091128182876,103.803120797076
1,7,STIRLING RESIDENCES,STIRLING ROAD,Apartment,03,RCR,99 yrs lease commencing from 2017,New Sale,1,1401000,-,635,Strata,36 to 40,2206,May-2021,1.29091128182876,103.803120797076
2,8,STIRLING RESIDENCES,STIRLING ROAD,Apartment,03,RCR,99 yrs lease commencing from 2017,New Sale,1,1578000,-,904,Strata,36 to 40,1745,May-2021,1.29091128182876,103.803120797076
3,119,STIRLING RESIDENCES,STIRLING ROAD,Apartment,03,RCR,99 yrs lease commencing from 2017,New Sale,1,1381000,-,635,Strata,36 to 40,2175,May-2021,1.29091128182876,103.803120797076
4,146,STIRLING RESIDENCES,STIRLING ROAD,Apartment,03,RCR,99 yrs lease commencing from 2017,New Sale,1,1007000,-,441,Strata,26 to 30,2282,May-2021,1.29091128182876,103.803120797076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96941,1311,MIMOSA PARK,MIMOSA CRESCENT,Condominium,28,OCR,Freehold,Resale,1,1780000,-,1755,Strata,01 to 05,1015,Apr-2019,1.38513290820973,103.862921421272
96942,1616,MIMOSA PARK,MIMOSA CRESCENT,Condominium,28,OCR,Freehold,Resale,1,1780000,-,1873,Strata,06 to 10,950,Aug-2018,1.38513290820973,103.862921421272
96943,1918,MIMOSA PARK,MIMOSA CRESCENT,Condominium,28,OCR,Freehold,Resale,1,1610000,-,1755,Strata,06 to 10,918,Apr-2018,1.38513290820973,103.862921421272
96944,2862,MIMOSA PARK,MIMOSA CRESCENT,Condominium,28,OCR,Freehold,Resale,1,1548888,-,1873,Strata,06 to 10,827,Aug-2017,1.38513290820973,103.862921421272


In [164]:
innerdata = pd.merge(df_merged, proplocation, left_index=True, right_index=True, how="outer")
innerdata

,S/N,ProjectName,StreetName_x,Type,PostalDistrict,MarketSegment,Tenure,TypeOfSale,NoUnits,Price,NettPrice,Sqft,TypeOfArea,FloorLevel,UnitPrice,Date of Sale,StreetName_y,latitude,longitude
0,1,STIRLING RESIDENCES,STIRLING ROAD,Apartment,03,RCR,99 yrs lease commencing from 2017,New Sale,1,1401000,-,635,Strata,36 to 40,2206,May-2021,STIRLING ROAD,1.29091128182876,103.803120797076
1,2,CLAVON,CLEMENTI AVENUE 1,Apartment,05,OCR,99 yrs lease commencing from 2019,New Sale,1,1831000,-,1130,Strata,11 to 15,1620,May-2021,CLEMENTI AVENUE 1,1.30819332208405,103.767915046417
2,3,CLAVON,CLEMENTI AVENUE 1,Apartment,05,OCR,99 yrs lease commencing from 2019,New Sale,1,2047000,-,1356,Strata,06 to 10,1509,May-2021,CHIN SWEE ROAD,1.28855135333783,103.841303134744
3,4,CLAVON,CLEMENTI AVENUE 1,Apartment,05,OCR,99 yrs lease commencing from 2019,New Sale,1,2077000,-,1356,Strata,11 to 15,1531,May-2021,SILAT AVENUE,1.27707706879078,103.831425856916
4,5,THE LANDMARK,CHIN SWEE ROAD,Condominium,03,RCR,99 yrs lease commencing from 2020,New Sale,1,1121000,-,495,Strata,16 to 20,2264,May-2021,BERNAM STREET,1.27386294350519,103.843710484506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96941,3,"For conventional landed housing, the unit tran...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96942,NaN,For strata sub-divided housing such as apartme...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96943,NaN,"To convert $ psf to $ psm, multiply by a facto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96944,4,New sale transactions starting from 25 May 201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
#data cleaning, dropping rows with strings

outerdata = outerdata[~outerdata['Price'].isin(['Price ($)'])] #drop rows with strings

outerdata["Price"]= pd.to_numeric(outerdata["Price"], downcast="float")
outerdata.iloc[19540:19547]


<ipython-input-168-5199651ace44>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outerdata["Price"]= pd.to_numeric(outerdata["Price"], downcast="float")


,S/N,ProjectName,StreetName,Type,PostalDistrict,MarketSegment,Tenure,TypeOfSale,NoUnits,Price,NettPrice,Sqft,TypeOfArea,FloorLevel,UnitPrice,Date of Sale,latitude,longitude
19540,Search By Period (MAY 2016 to MAY 2021) & Prop...,02,03,04,05 & Type of Sale: New Sale,Sub Sale,Resale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.29746933994517,103.823538741274
19546,1,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,1374000.0,-,495,Strata,26 to 30,2775,May-2021,1.29693762042782,103.831387750972
19547,7,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,1806000.0,-,657,Strata,16 to 20,2751,May-2021,1.29693762042782,103.831387750972
19548,26,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,1368000.0,-,495,Strata,26 to 30,2763,May-2021,1.29693762042782,103.831387750972
19549,49,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,2328000.0,-,861,Strata,21 to 25,2703,May-2021,1.29693762042782,103.831387750972
19550,59,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,1792000.0,-,657,Strata,16 to 20,2729,May-2021,1.29693762042782,103.831387750972
19551,65,IRWELL HILL RESIDENCES,IRWELL HILL,Apartment,09,CCR,99 yrs lease commencing from 2020,New Sale,1,1678000.0,-,624,Strata,16 to 20,2688,May-2021,1.29693762042782,103.831387750972


In [225]:
outermean = outerdata.groupby("StreetName", as_index=False)["Price"].mean()
outermean.iloc[600:650]

mapdata = pd.merge(outermean, proplocation, on=["StreetName"])
mapdata = mapdata.drop([mapdata.index[0]])
mapdata = mapdata.drop([mapdata.index[233]])
mapdata = mapdata.drop([mapdata.index[304]])
mapdata = mapdata.drop([mapdata.index[709]]) #dropping value that are not found
mapdata["latitude"]= pd.to_numeric(mapdata["latitude"], downcast="float")
mapdata["longitude"]= pd.to_numeric(mapdata["longitude"], downcast="float") #to enable folium to work, convert from object
mapdata.dtypes
# why are some of the street names missing?

StreetName     object
Price         float32
latitude      float32
longitude     float32
dtype: object

In [227]:
address = 'Singapore, Singapore'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


In [228]:
map_singapore = folium.Map(location=[latitude, longitude], zoom_start=11)


for latitude, longitude, StreetName, Price in zip(mapdata['latitude'], mapdata['longitude'], mapdata['StreetName'], mapdata['Price']):
    label = '{}, {}'.format(Price, StreetName)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_singapore)  
    
map_singapore

<h3>Notice that the Neiborhoods are very evenly spaced out.</h3>

In [ ]:
CLIENT_ID = 'K4YZ0DCCW2EWL5RQQTVEP0FU4C43H3JOWDIYETH2XDMSN45C' 
CLIENT_SECRET = '' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
table=soup.find_all('table')

table


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df